In [40]:
import random
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_csv('../../data/clean/dfu.csv')

In [6]:
df2=df[['price','odometer','manufacturer', 'model', 'condition',
       'cylinders', 'size','fuel','title', 'transmission', 'drive', 'type', 'color']]

In [7]:
df1_final = pd.get_dummies(df2, drop_first=True)
print(df1_final.columns)

Index(['price', 'odometer', 'manufacturer_audi', 'manufacturer_bmw',
       'manufacturer_buick', 'manufacturer_cadillac', 'manufacturer_chevrolet',
       'manufacturer_chrysler', 'manufacturer_datsun', 'manufacturer_dodge',
       ...
       'color_brown', 'color_custom', 'color_green', 'color_grey',
       'color_orange', 'color_purple', 'color_red', 'color_silver',
       'color_white', 'color_yellow'],
      dtype='object', length=725)


In [17]:
X_head = df1_final.iloc[:, df1_final.columns != 'price']
X = df1_final.loc[:, df1_final.columns != 'price']
y = df1_final['price']
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=1)

In [19]:
dtr = DecisionTreeRegressor(random_state=1)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [47]:
test_predictions = dtr.predict(X_test)
test_mae = mean_absolute_error(test_predictions, y_test)
print("MAE: {:,.0f}".format(test_mae))

MAE: 7,308


In [48]:
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(X_train, y_train)
    preds_test = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds_test)
    return(mae)

In [49]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
scores = {leaf_size: get_mae(leaf_size, X_train, X_test, y_train, y_test) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)

In [32]:
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)
final_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=500, random_state=1)

In [53]:
test_predictions = final_model.predict(X_test)
test_mae = mean_absolute_error(test_predictions, y_test)
print("MAE: {:,.0f}".format(test_mae))

MAE: 460


 random forest

In [54]:
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(X_train, y_train)
rf_test_predictions = rf_model.predict(X_test)
rf_test_mae = mean_absolute_error(rf_test_predictions, y_test)
print("MAE: {:,.0f}".format(rf_test_mae))

MAE: 3,502


In [55]:
scores = []
forest = RandomForestRegressor(n_estimators=20, random_state=0)
acc = cross_val_score(forest, X_train, y_train, scoring='r2', cv=5)
scores.append(round(acc.mean()*100,2))

In [56]:
results = pd.DataFrame({
    'Metrics': ['R2'],
    'Accuracy': scores})
results

,Metrics,Accuracy
0,R2,71.32
